In [1]:
!nvidia-smi

Mon May 22 19:37:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.89.02    Driver Version: 525.89.02    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:01:00.0 Off |                    0 |
| N/A   36C    P0    36W / 250W |  22723MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  Off  | 00000000:24:00.0 Off |                    0 |
| N/A   

In [2]:
!export CUDA_VISIBLE_DEVICES=4

In [3]:
!echo $CUDA_VISIBLE_DEVICES

In [4]:
import glob
from torch.utils.data import DataLoader
from custom_dataset_cross import MyDataset
import cv2
from share import *
import torchvision
import einops
import numpy as np
from ldm.modules.encoders.modules import FrozenClipImageEmbedder
import torch.nn.functional as F
import config
import gradio as gr
import random
import torch
from PIL import Image
from pytorch_lightning import seed_everything
from annotator.util import resize_image, HWC3
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler

logging improved.


/export/home/nstracke/miniconda3/envs/control/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/export/home/nstracke/miniconda3/envs/control/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


In [5]:
seed = -1
if seed == -1:
    seed = random.randint(0, 65535)
seed_everything(seed)

if config.save_memory:
    model.low_vram_shift(is_diffusing=False)

Global seed set to 64401


In [6]:
def get_key(name: str):
    name = name.split('.')[1] 
    name = name.split('_')[-1]
    name = name[3:]
    return int(name)

def get_sequence():
    for batch in dataloader:
        meta_batch = batch['meta']['file_name']
        meta = meta_batch[0]
        meta = meta.split('/')[1]
        meta = meta.split('_')[:-1]
        meta = '_'.join(meta)
        print(meta)
        styles = glob.glob('./data/kin_hed2/jpg/' + meta + '*')
        styles.sort(key=get_key)
        structures = glob.glob('./data/kin_hed2/hint/' + meta + '*')
        structures.sort(key=get_key)
        print(styles)
        print(structures)
        return styles, structures

def get_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # img = img.astype(np.float32) / 255.0
    return img

In [7]:
def make_grid(batch, length, path):
    grid = torchvision.utils.make_grid(batch, nrow=length)
    grid = grid.transpose(0, 1).transpose(1, 2).squeeze(-1)
    grid = grid.cpu().numpy().clip(0, 255).astype(np.uint8)
    Image.fromarray(grid).save(path)


In [8]:
dataset = MyDataset('kin_hed2')
dataloader = DataLoader(dataset, num_workers=16, batch_size=1, shuffle=True)

In [9]:
p1 = './train_log/kin_hed_2/lightning_logs/version_0/checkpoints/epoch=1-step=119930.ckpt'
p2 = './train_log/kin_hed_cross_3/lightning_logs/version_0/checkpoints/epoch=4-step=412223.ckpt'

print('loading model 1...')
model = create_model('./models/cldm_v15_org.yaml').cpu()
model.load_state_dict(load_state_dict(p1, location='cuda'))
model = model.cuda()

print('loading model 2...')
model2 = create_model('./models/cldm_v15_cross.yaml').cpu()
model2.load_state_dict(load_state_dict(p2, location='cuda'))
model2 = model.cuda()



ddim_sampler = DDIMSampler(model)
frozenClipImageEmbedder = FrozenClipImageEmbedder()
frozenClipImageEmbedder = frozenClipImageEmbedder.cuda()


loading model 1...


/export/home/nstracke/miniconda3/envs/control/lib/python3.8/site-packages/torch/distributed/_sharded_tensor/__init__.py:8: DeprecationWarning: torch.distributed._sharded_tensor will be deprecated, use torch.distributed._shard.sharded_tensor instead
  warnings.warn(


ControlLDM: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_

In [10]:
!export CUDA_VISIBLE_DEVICES=4

ddim_steps = 40
strength = 1
eta = 0

for idx in range(10):
    styles, structures = get_sequence()
    styles = np.array([get_img(style) for style in styles])
    styles = styles.copy().astype(np.float32)

    structures = np.array([get_img(structure) for structure in structures])
    structures = structures.astype(np.float32) / 255.0

    B, H, W, C = structures.shape
    shape = (4, H // 8, W // 8)


    control = torch.from_numpy(structures.copy()).float().cuda()
    control = einops.rearrange(control, 'b h w c -> b c h w')

    style = frozenClipImageEmbedder(styles)
    c_style = style.last_hidden_state
    c_embed = style.pooler_output  

    uc_style =  c_style # torch.zeros_like(c_style) #
    uc_embed = c_embed

    # 11111111111111111111111111111111

    scale = 2


    cond = {"c_concat": [control], "c_crossattn": [model.get_learned_conditioning(["a professional, detailed, high-quality image"]*B)], "c_style": [c_style], "c_embed": [c_embed] }
    un_cond = {"c_concat": [control], "c_crossattn": [model.get_learned_conditioning(['']*B)], "c_style": [uc_style], "c_embed": [uc_embed]  }

    model.control_scales =  ([strength] * 13)  # Magic number. ID
    samples, intermediates = ddim_sampler.sample(ddim_steps, B,
                                                shape, cond, verbose=False, eta=eta,
                                                unconditional_guidance_scale=scale,
                                                unconditional_conditioning=un_cond)

    x_samples = model.decode_first_stage(samples)
    x_samples = x_samples * 127.5 + 127.5
    # ==========================================

    # 2222222222222222222222222222222222222

    scale = 2

    cond = {"c_concat": [control], "c_crossattn": [model2.get_learned_conditioning(["a professional, detailed, high-quality image"]*B)], "c_style": [c_style], "c_embed": [c_embed] }
    un_cond = {"c_concat": [control], "c_crossattn": [model2.get_learned_conditioning(['']*B)], "c_style": [uc_style], "c_embed": [uc_embed]  }

    model2.control_scales =  ([strength] * 13)  # Magic number. ID
    samples, intermediates = ddim_sampler.sample(ddim_steps, B,
                                                shape, cond, verbose=False, eta=eta,
                                                unconditional_guidance_scale=scale,
                                                unconditional_conditioning=un_cond)

    x_samples2 = model2.decode_first_stage(samples)
    x_samples2 = x_samples2 * 127.5 + 127.5
    # ==========================================









    ss = einops.rearrange(styles, 'b h w c -> b c h w')
    ss = ss[1:]
    ss = torch.Tensor(ss)
    ss = ss.cuda()
    path = 'vid_check/' + str(idx) + '.jpg'
    make_grid(torch.cat((ss, x_samples, x_samples2)), B, path)

fEkBAEqJXWk_000021_000031
['./data/kin_hed2/jpg/fEkBAEqJXWk_000021_000031_idx0.png', './data/kin_hed2/jpg/fEkBAEqJXWk_000021_000031_idx500.png', './data/kin_hed2/jpg/fEkBAEqJXWk_000021_000031_idx1000.png', './data/kin_hed2/jpg/fEkBAEqJXWk_000021_000031_idx1500.png', './data/kin_hed2/jpg/fEkBAEqJXWk_000021_000031_idx2000.png']
['./data/kin_hed2/hint/fEkBAEqJXWk_000021_000031_idx500.png', './data/kin_hed2/hint/fEkBAEqJXWk_000021_000031_idx1000.png', './data/kin_hed2/hint/fEkBAEqJXWk_000021_000031_idx1500.png', './data/kin_hed2/hint/fEkBAEqJXWk_000021_000031_idx2000.png']
Data shape for DDIM sampling is (4, 4, 64, 64), eta 0
Running DDIM Sampling with 40 timesteps


DDIM Sampler: 100%|██████████| 40/40 [00:16<00:00,  2.43it/s]


Data shape for DDIM sampling is (4, 4, 64, 64), eta 0
Running DDIM Sampling with 40 timesteps


DDIM Sampler: 100%|██████████| 40/40 [00:16<00:00,  2.43it/s]


QgIXLpEwuLM_000015_000025
['./data/kin_hed2/jpg/QgIXLpEwuLM_000015_000025_idx0.png', './data/kin_hed2/jpg/QgIXLpEwuLM_000015_000025_idx500.png', './data/kin_hed2/jpg/QgIXLpEwuLM_000015_000025_idx1000.png', './data/kin_hed2/jpg/QgIXLpEwuLM_000015_000025_idx1500.png', './data/kin_hed2/jpg/QgIXLpEwuLM_000015_000025_idx2000.png']
['./data/kin_hed2/hint/QgIXLpEwuLM_000015_000025_idx500.png', './data/kin_hed2/hint/QgIXLpEwuLM_000015_000025_idx1000.png', './data/kin_hed2/hint/QgIXLpEwuLM_000015_000025_idx1500.png', './data/kin_hed2/hint/QgIXLpEwuLM_000015_000025_idx2000.png']
Data shape for DDIM sampling is (4, 4, 64, 64), eta 0
Running DDIM Sampling with 40 timesteps


...

DDIM Sampler: 100%|██████████| 40/40 [00:16<00:00,  2.38it/s]


HDjZzu0bpaU_000002_000012
['./data/kin_hed2/jpg/HDjZzu0bpaU_000002_000012_idx0.png', './data/kin_hed2/jpg/HDjZzu0bpaU_000002_000012_idx500.png', './data/kin_hed2/jpg/HDjZzu0bpaU_000002_000012_idx1000.png', './data/kin_hed2/jpg/HDjZzu0bpaU_000002_000012_idx1500.png', './data/kin_hed2/jpg/HDjZzu0bpaU_000002_000012_idx2000.png']
['./data/kin_hed2/hint/HDjZzu0bpaU_000002_000012_idx500.png', './data/kin_hed2/hint/HDjZzu0bpaU_000002_000012_idx1000.png', './data/kin_hed2/hint/HDjZzu0bpaU_000002_000012_idx1500.png', './data/kin_hed2/hint/HDjZzu0bpaU_000002_000012_idx2000.png']
Data shape for DDIM sampling is (4, 4, 64, 64), eta 0
Running DDIM Sampling with 40 timesteps


DDIM Sampler: 100%|██████████| 40/40 [00:16<00:00,  2.38it/s]


Data shape for DDIM sampling is (4, 4, 64, 64), eta 0
Running DDIM Sampling with 40 timesteps


DDIM Sampler: 100%|██████████| 40/40 [00:16<00:00,  2.38it/s]
